In [8]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from split_asl_alphabet.dataset import ASLDataset
from ResNet import Bottleneck, ResNet, ResNet50

In [9]:
transform = transforms.Compose([
    transforms.ToPILImage(),  # Assuming you're starting with a tensor
    transforms.RandomResizedCrop(size=(226, 226), antialias=True),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Gray normalization
])

In [11]:

train_dir = "asl_alphabet_train"

train_data_simple = ASLDataset(data_path=train_dir,range_index = [1,2500], transform=transform)
test_data_simple = ASLDataset(data_path=train_dir,range_index = [2500,3000], transform=transform)

# 2. Turn data into DataLoaders
import os
from torch.utils.data import DataLoader

# Setup batch size and number of workers 
BATCH_SIZE = 1
NUM_WORKERS = os.cpu_count()
print(f"Creating DataLoader's with batch size {BATCH_SIZE} and {NUM_WORKERS} workers.")

# Create DataLoader's
train_dataloader_simple = DataLoader(train_data_simple, 
                                     batch_size=BATCH_SIZE, 
                                     shuffle=True, 
                                     num_workers=NUM_WORKERS)

test_dataloader_simple = DataLoader(test_data_simple, 
                                    batch_size=BATCH_SIZE, 
                                    shuffle=False, 
                                    num_workers=NUM_WORKERS)
print(f'train dataset size :{len(train_data_simple)} images ')
print(f'test dataset size :{len(test_data_simple)} images ')

Creating DataLoader's with batch size 1 and 20 workers.


AttributeError: 'ASLDataset' object has no attribute 'img_paths'

In [ ]:
net = ResNet50(29).to('cpu')

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.1, patience=5)

In [ ]:
EPOCHS = 10
for epoch in range(EPOCHS):
    losses = []
    running_loss = 0
    for i, inp in enumerate(train_dataloader_simple):
        inputs= inp
        inputs, labels = inputs.to('cpu'), labels.to('cpu')
        optimizer.zero_grad()
    
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i%100 == 0 and i > 0:
            print(f'Loss [{epoch+1}, {i}](epoch, minibatch): ', running_loss)
            running_loss = 0.0

    avg_loss = sum(losses)/len(losses)
    scheduler.step(avg_loss)
            
print('Training Done')

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for data in test_dataloader_simple:
        images, labels = data
        images, labels = images.to('cuda'), labels.to('cuda')
        outputs = net(images)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy on 10,000 test images: ', 100*(correct/total), '%')

Accuracy on 10,000 test images:  85.66 %
